#**Model Attempt #2**

Improvments To Be Made:

Data Augmentation- RandomHorizontalFlip, RandomRotation, ColorJitter

Make Image BIGGER- 64×64 to 128×128

CNN Update- Add BatchNorm2d after conv layers

Dropout- added 0.5 before final fully connected layer

Learning Rate Scheduler- ReduceLROnPlateau

Lengthen Training and Early Stopping- stop if no improvement in 5 epochs

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
import random
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.image import imread
import os
from imageio import imread

import warnings
warnings.filterwarnings('ignore')

#from google.colab import drive
#drive.mount('/content/drive')
#!unzip /content/drive/MyDrive/dataset.zip -d /content/dataset
#path = '/content/dataset/dataset'
path = './dataset'  # Update this path as needed


In [6]:
#initialize
width = []
height = []
channel_color = []
weather_type = []

#set up path
for folder in os.listdir(path):
    subfolder_path = os.path.join(path, folder)

    for image_file in os.listdir(subfolder_path):
        image_path = os.path.join(subfolder_path, image_file)

        image = imread(image_path)

        if len(image.shape) < 3:
            image = image.reshape(image.shape + (1,))

        h, w, c = image.shape

        #update lists
        width.append(w)
        height.append(h)
        channel_color.append(c)
        weather_type.append(folder)

#make lists a df
df = pd.DataFrame({'width': width, 'height': height, 'channels': channel_color, 'weather': weather_type})
df.head()

,width,height,channels,weather
0,400,283,3,lightning
1,400,300,3,lightning
2,400,264,3,lightning
3,378,188,3,lightning
4,284,238,3,lightning


# **Model**

In [9]:
import torch.nn.functional as F
from collections import Counter
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])


dataset = datasets.ImageFolder('./dataset', transform=transform) #Set to your correct dataset path!!
num_classes = len(dataset.classes)
print("Classes:", dataset.classes)


train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 32 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = SimpleCNN(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_losses = []

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

epochs = 20
best_val_acc = 0
best_model_wts = copy.deepcopy(model.state_dict())
patience_counter = 0
early_stop_patience = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    correct = 0
    total = 0
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs} | "
    f"Train Loss: {avg_train_loss:.4f} | "
    f"Val Loss: {avg_val_loss:.4f} | "
    f"Val Acc: {val_acc:.2f}%")

    scheduler.step(avg_val_loss)
    print(f"Current LR: {optimizer.param_groups[0]['lr']}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stop_patience:
            print("Early stopping triggered")
            break

model.load_state_dict(best_model_wts)
print(f"Best Validation Accuracy: {best_val_acc:.2f}%")


Using device: cpu
Classes: ['dew', 'fogsmog', 'frost', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime', 'sandstorm', 'snow']
Epoch 1/20 | Train Loss: 2.3866 | Val Loss: 1.7558 | Val Acc: 45.81%
Current LR: 0.001
Epoch 2/20 | Train Loss: 1.9474 | Val Loss: 1.6676 | Val Acc: 50.11%
Current LR: 0.001
Epoch 3/20 | Train Loss: 1.8729 | Val Loss: 1.5059 | Val Acc: 52.59%
Current LR: 0.001
Epoch 4/20 | Train Loss: 1.7934 | Val Loss: 1.3708 | Val Acc: 51.86%
Current LR: 0.001
Epoch 5/20 | Train Loss: 1.7775 | Val Loss: 1.3516 | Val Acc: 54.12%
Current LR: 0.001
Epoch 6/20 | Train Loss: 1.7367 | Val Loss: 1.3777 | Val Acc: 53.75%
Current LR: 0.001
Epoch 7/20 | Train Loss: 1.6959 | Val Loss: 1.3059 | Val Acc: 56.96%
Current LR: 0.001
Epoch 8/20 | Train Loss: 1.6929 | Val Loss: 1.3620 | Val Acc: 53.53%
Current LR: 0.001
Epoch 9/20 | Train Loss: 1.6714 | Val Loss: 1.2646 | Val Acc: 55.94%
Current LR: 0.001
Epoch 10/20 | Train Loss: 1.6474 | Val Loss: 1.2517 | Val Acc: 56.88%
Current LR: 0.